# Setup

In [1]:
import csv
from datetime import datetime
import json
import os
import pprint
from Queue import Queue
import string
from threading import Thread

from ipywidgets import FloatProgress
from IPython.display import display
import numpy as np
import pandas as pd

In [2]:
src = '../../../data/src/rate-beer/'

# Load Source Data

In [3]:
# load data to dataframe
fname = os.path.abspath(os.path.join(src, 'beer.csv'))
data = pd.read_csv(fname)

In [4]:
# remove unnecessary columns
data = data.iloc[:,0:20]
del data['_has_fetched']

In [5]:
data.id += 1

In [6]:
for i,c in enumerate(data.columns):
    print('{}: {}'.format(i,c))

0: id
1: abv
2: brewed_at
3: brewery
4: calories
5: description
6: ibu
7: img_url
8: mean_rating
9: name
10: num_ratings
11: overall_rating
12: retired
13: seasonal
14: style
15: style_rating
16: style_url
17: tags
18: url


# Parse Data

## Create DataFrames

In [7]:
beers = pd.DataFrame(columns=data.columns)
del beers['tags']
tags = pd.DataFrame(columns=['id', 'name'])
beer_tag = pd.DataFrame(columns=['beer_id', 'tag_id'])

In [8]:
# global variables
num_fetch_threads = 10
queue = Queue()
tag_id = 0

# build queue
ids = range(1, data.id.max() + 1)
for id in ids:
    q = id
    queue.put(q)

# set up progress bar
p = FloatProgress(min=0, max=queue.qsize(), description="Parsing pages")
display(p)

#define worker
def parse_data(i, q):
    global p
    global tags
    global beer_tag
    global tag_id
    while True:
        id = q.get()
        row = data[data.id==id].iloc[0]
        try:
            if row.tags != '[]':
                tag_list =row.tags.strip('[').strip(']')
                tag_list = tag_list.split(',')
                tag_list = [t.strip() for t in tag_list]
                for tag in tag_list:
                    current_tags = tags.name.tolist()
                    if not tag in current_tags:
                        tag_id += 1
                        new_tag = pd.DataFrame(dict(id=tag_id, name=tag), index=[0])
                        tags = tags.append(new_tag)

                        new_beer_tag = pd.DataFrame(dict(beer_id=row.id, tag_id=tag_id), index=[0])
                        beer_tag = beer_tag.append(new_beer_tag)
                    else:
                        current_tag = tags[tags.name==tag].iloc[0].id
                        new_beer_tag = pd.DataFrame(dict(beer_id=row.id, tag_id=current_tag), index=[0])
                        beer_tag = beer_tag.append(new_beer_tag)
        except:
            print row.tags
            pass
            
        p.value += 1
        q.task_done()
        
# open threads
for i in range(num_fetch_threads):
    worker = Thread(target=parse_data, args=(i, queue,))
    worker.setDaemon(True)
    worker.start()

# start task queue
queue.join()

[rauchbock, smoked malt, bamberg smoked malt]
nan
[table beer, brettanomyces]
nan
[nefiltrovan??, saaz, 12??, cervenak, cervenak]


In [15]:
wrk = '../../../data/wrk/ratebeer/'
# save tags
fname = os.path.abspath(os.path.join(wrk, 'tags.csv'))
tags.to_csv(fname, quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', index=False)

# save beers
del data['tags']
fname = os.path.abspath(os.path.join(wrk, 'beers.csv'))
data.to_csv(fname, quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', index=False)

# save beer_tags
fname = os.path.abspath(os.path.join(wrk, 'beer_tag.csv'))
beer_tag.to_csv(fname, quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', index=False)

In [11]:
tags.shape

(682, 2)